In [2]:
!pip install selenium
!pip install webdriver-manager

In [47]:
# this library is to manipulate browser
from selenium import webdriver

# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import unidecode

First we go to the ONPE website 

In [48]:
# Ejecute driver
service = Service(executable_path="C:\\Users\\sofi\\Documents\\GitHub\\resumenpython\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.maximize_window()

url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'
driver.get(url)
time.sleep(3)

Then we go to Resumen general 

In [49]:
resumen_general = driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]')
resumen_general.click()
time.sleep(3)

Then we go to Elecciones presidenciales

In [50]:
presidential_peru = driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a')
presidential_peru.click()
time.sleep(3)

After that we open Peru

In [51]:
peru = driver.find_element( By.XPATH  , '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]')
peru.click()


After that we scrape the data 

In [54]:
# Store all_tables
all_tables = {}

# we loop over all departments
for dpt_idx in range( 26 ):
    try:

        # Get again all departments since HTML is refreshing
        # all elements
        # Click on one specific department
        dpt = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
        department = dpt.options[ dpt_idx ]

        # Get departmant name
        dpt_name = department.text

        # We select a different department name
        if dpt_name != "--TODOS--" :

            # click on department
            department.click()

            # Get all elements of province
            prov = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            num_prov_options = len( prov.options )

            for prov_idx in range( num_prov_options ):
                try:

                    # Get again all districts since HTML is refreshing
                    # all elements
                    prov = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
                    province = prov.options[ prov_idx ]

                    # Get province name
                    prov_name = province.text

                    if prov_name != "--TODOS--" :

                        # click on province
                        province.click()

                        # Get all elements from district
                        dist = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                        num_dist_options = len( dist.options )

                        for dist_idx in range( num_dist_options ):
                            try:

                                # Get again all districts since HTML is refreshing
                                # all elements
                                dist = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                                district = dist.options[ dist_idx ]

                                # Get district name
                                dist_name = district.text

                                if dist_name != "-- SELECCIONE --" :

                                    # click on district
                                    district.click()

                                    # Get UBIGEO
                                    ubigeo = driver.current_url.split("/")[ -1 ]

                                    ## Gbet table of presidential votes
                                    # Get html at this point
                                    table_path = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
                                    table_html = table_path.get_attribute( 'innerHTML' )
                                    # Read the table using pandas
                                    table = pd.read_html( table_html )

                                    # Cleaning tables
                                    row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                                    clean_columns = row_new_columns \
                                                          .str.replace( " ", "_") \
                                                          .str.lower().str.replace( "%", "share_") \
                                                          .apply( lambda x : unidecode.unidecode( x ) ) \
                                                          .tolist()

                                    # Selecting specific columns
                                    table_clean = table[0].iloc[ 1:, 2: ].copy()

                                    # rename columns
                                    table_clean.columns = clean_columns

                                    # New values to columns 
                                    table_clean[ 'department' ] = dpt_name
                                    table_clean[ 'province' ]   = prov_name
                                    table_clean[ 'district' ]   = dist_name


                                    # store tables
                                    all_tables[ ubigeo ] = table_clean
                                    
                            except:
                                print(f"error in {dist_idx}")
                                pass
                except:
                    print(f"error in {prov_idx}")
                    pass
    except:
        print(f"error in {dpt_idx}")
        pass 

error in 1
error in 2
error in 1
error in 2
error in 2
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 1
error in 2
error in 6
error in 1
error in 1


In [55]:
final_table = pd.concat( all_tables.values() ).reset_index( drop = True )


In [56]:
final_table

,organizaciones_politicas,total,share_validos,share_emitidos,department,province,district
0,PARTIDO NACIONALISTA PERUANO,356,9.327%,6.389%,AMAZONAS,BAGUA,BAGUA
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,0.341%,0.233%,AMAZONAS,BAGUA,BAGUA
2,PARTIDO MORADO,41,1.074%,0.736%,AMAZONAS,BAGUA,BAGUA
3,PERU PATRIA SEGURA,4,0.105%,0.072%,AMAZONAS,BAGUA,BAGUA
4,VICTORIA NACIONAL,151,3.956%,2.710%,AMAZONAS,BAGUA,BAGUA
...,...,...,...,...,...,...,...
36801,ALIANZA PARA EL PROGRESO,847,7.935%,5.352%,UCAYALI,PADRE ABAD,PADRE ABAD
36802,TOTAL DE VOTOS VÁLIDOS,10674,100.000%,67.446%,UCAYALI,PADRE ABAD,PADRE ABAD
36803,VOTOS EN BLANCO,3910,NaN,24.706%,UCAYALI,PADRE ABAD,PADRE ABAD
36804,VOTOS NULOS,1242,NaN,7.848%,UCAYALI,PADRE ABAD,PADRE ABAD
